### Copyright 2022 Edward Späth, Frankfurt University of Applied Sciences, FB2, Computer Science
### No liability or warranty; only for educational and non-commercial purposes
### See some basic hints for working with Jupyter notebooks in README.md

## Dijkstra's pathfinding algorithm without visualization

## Data structure for storing node info

In [1]:
class Node:
    node_id = 0
    def __init__(self, name_input):
        self.name = name_input
        self.predecessor = 'Cannot be accessed'
        self.distance = float('inf') # Positive infinity
        self.adjacencylist = []
        my_dict[self.name] = Node.node_id
        Node.node_id += 1

## Global variables for storing information

In [2]:
my_dict = {} # for easier access, each node's name returns said node's index in the list if hashed. 
weight_dict = {} # for storing weights
nodes = []
visited_nodes = []

## Functions for resetting results or the entire graph

In [3]:
def reset_results(): # Resets data after results were printed
    global visited_nodes
    visited_nodes = []
    # Set node statistics back to initial values.
    for node in nodes:  
        node.predecessor = 'Cannot be accessed'
        node.distance = 'Infinity'

In [4]:
def reset_graph():
    global nodes, my_dict, weight_dict, visited_nodes
    visited_nodes = []
    nodes = []
    my_dict = {}
    weight_dict = {}
    Node.node_id = 0

## Functions for creating a graph

In [5]:
def create_graph(nodearray, edgearray):
    add_nodes(nodearray)
    add_edges(edgearray)

def add_nodes(nodearray):
    global nodes
    for node_name in nodearray:
        if my_dict.get(node_name) is None: # If node was not added already
            nodes.append(Node(node_name))

def add_edges(edgearray):
    weighted_edgearray = []
    for my_tuple in edgearray:
        if len(my_tuple) == 2: # len(my_tuple) is 2 when only start and destination are in it, meaning weight was left out. In that case add a weight of 1
            weighted_edgearray.append((my_tuple[0], my_tuple[1], 1))
        else: # Weights already present in input 
            weighted_edgearray.append((my_tuple[0], my_tuple[1], my_tuple[2])) 
    for start, dest, weight in weighted_edgearray:
        # In case it is not a weighted graph and user has put random values as weights
        start_index = my_dict.get(start) # Access to index of node element instead of name
        dest_index = my_dict.get(dest)
        # If my_dict.get(x) returns None that means that x was not found in the hashmap, implying the node was not inserted and does not exist or else it would be in the hashmap
        if start_index is not None and dest_index is not None:
            # If this edge was not added before (multiple edges facing the same direction between two nodes is forbidden here)
            if dest not in nodes[start_index].adjacencylist:
                # Each node is given a unique id at the time of insertion. By sorting the adjacencylist according to this unique id,
                # you can define which node is chosen when there are multiple options available.
                # Having nodearray = ['A', 'B', 'C' ...] gurantees that if given a "choice", the algorithm will always visit 'A' over 'B', 'B' over 'C' and 'A' over 'C'
                at_index = len(nodes[start_index].adjacencylist)
                for index, adjacent in enumerate(nodes[start_index].adjacencylist):
                    if my_dict[adjacent.name] > my_dict[dest]:
                        at_index = index
                        break
                nodes[start_index].adjacencylist.insert(at_index, nodes[my_dict[dest]])
                weight_dict[str(start) + str(dest)] = weight

## Priority Queue implementation

In [6]:
import copy
class PriorityQueue():
    def __init__(self):
        self.heapsize = len(nodes)
        # The elements of the priority queue are deep copies of global "nodes" list. Else the order of nodes would be changed, 
        # leading to problems with hashing and having it in a different order than the user has input
        self.elements = copy.deepcopy(nodes)
        # No need for heap_decrease_key as priority is infinity for all elements 

    def is_empty(self):
        return self.heapsize == 0
            
    def parent(self, i):
        return (i-1)//2

    def left(self, i):
        return 2*i+1

    def right(self, i):
        return 2*i+2

    def heap_decrease_key(self, i, distance_input):
        self.elements[i].distance = distance_input   
        while i > 0 and self.elements[self.parent(i)].distance > self.elements[i].distance:
            self.elements[i], self.elements[self.parent(i)] = self.elements[self.parent(i)], self.elements[i]
            i = self.parent(i)

    def extract_min(self):   
        min_element = self.elements[0]
        self.heapsize -= 1
        self.elements[0], self.elements[self.heapsize] = self.elements[self.heapsize], self.elements[0] # Swaps first element and last element to delete the previous first and bubble down the previous last
        self.elements = self.elements[:self.heapsize] # Deletes last element
        self.min_heapify(0)
        return min_element

    def min_heapify(self, i):
        l = self.left(i)
        r = self.right(i)   
        if l < self.heapsize and self.elements[l].distance < self.elements[i].distance:
            minimum = l
        else:
            minimum = i       
        if r < self.heapsize and self.elements[r].distance < self.elements[minimum].distance:
            minimum = r   
        if minimum != i:
            self.elements[i], self.elements[minimum] = self.elements[minimum], self.elements[i]
            self.min_heapify(minimum)

    def reduce_distance(self, element_to_be_searched, distance_input):   
        global nodes
        for index, element in enumerate(self.elements):
            if element.name == element_to_be_searched.name:
                at_index = index
                break
        self.heap_decrease_key(at_index, distance_input)
        nodes[my_dict[element_to_be_searched.name]].distance = distance_input

## Dijkstra algorithm

In [7]:
def dijkstra(s_name):
    global visited_nodes
    s_index = my_dict.get(s_name)
    if s_index is None:
        print("\nERROR: The Starting Node", s_name, "does not exist. Please make sure you have given the correct name to the start variable")
        return
    s = nodes[s_index]
    visited_nodes = []
    Q = PriorityQueue()
    Q.reduce_distance(s, 0)
    s.predecessor = 'NIL'
    while not Q.is_empty():
        u = Q.extract_min()
        visited_nodes.append(u.name)
        print("Currently visiting", u.name)
        for ν in u.adjacencylist:
            if ν.name not in visited_nodes:
                relax(u, ν, Q)
        print("\tNode", u.name, "has been finished\n")
    print_results()

def relax(u, ν, Q):
    distance_from_u_to_ν = weight_dict[str(u.name) + str(ν.name)]
    if ν.distance > u.distance + distance_from_u_to_ν:
        print("\tUpdated distance of node", ν.name, "from previously", ν.distance, "to", u.distance + distance_from_u_to_ν)
        Q.reduce_distance(ν, u.distance + distance_from_u_to_ν)
        # Set u to be the predecessor of ν
        set_predecessor(ν, u)

# Function for abstracting the setting of the predecessor away from the algorithm code itself
def set_predecessor(ν, u):
    nodes[my_dict[ν.name]].predecessor = nodes[my_dict[u.name]]
    
def show_shortest_path(start_name, destination_name):
    # destination_name == None <--> user does not want shortest path to be shown
    if destination_name == None:
        return
    start_index = my_dict.get(start_name)
    destination_index = my_dict.get(destination_name)   
    if start_index is None or destination_index is None:
        print("\nERROR: The starting node or target node does not exist (show_shortest_path)")
        return
    if destination_name == start_name:
        print("\nThe path from a node to itself is trivial")
        return
    destination = nodes[destination_index]
    if destination.predecessor == 'Cannot be accessed':
        print("\nThere is no connection between the starting node", start_name, "and the target", destination_name)
        return
    path = []
    while True:
        path.append(destination.name)
        if destination.name == start_name:
            break
        destination = destination.predecessor
    path.reverse()
    path_with_arrows = ''
    for i in range(len(path)):
        path_with_arrows += str(path[i])
        if i != len(path)-1:
            path_with_arrows += ' --> '
    print("The path to get from", start_name, "to", destination_name, "is:")
    print(path_with_arrows)

## Function to print results

In [8]:
def print_results():
    print("The algorithm is over")
    print("The nodes were visited in the following order:\n", visited_nodes)
    print("Here is each node's data:\n")
    for node in nodes:
        print("\tName:", node.name)
        # In case the predecessor is 'NIL' or 'Cannot be accessed', you have to print it as a string
        if isinstance(node.predecessor, str):
            print("\tPredecessor:", node.predecessor)
        # If the predecessor is not 'NIL' or 'Cannot be accessed', you have to print the node's name attribute
        else:
            print("\tPredecessor:", node.predecessor.name)
        print("\tDistance:", node.distance, "\n")

## Example

In [9]:
# Input the names of the nodes here. Regardless of edge input method!
# Nodes will be visited in the order of how they are given in nodearray if multiple options exist

nodearray = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

# All edges are directed. If you want a visualized version, see "Dijkstra.ipynb" in the same GitHub folder as this file

# It is a list (array) of 3-tuples. Left and middle components are strings refering to the node names.
# Left is the start of an edge and the right is the destination.
# The right value represents the distance between the start and the destination
# Syntax: edgearray = [('START', 'DESTINATION', WEIGHT), ('STARTx', 'DESTINATIONy', WEIGHTz), ...]
# Alternatively you can just leave out the weight altogether. Doing this will result in the weight automatically being turned to 1
# edgearray = [('A', 'B'), ('B', 'C')] is the same as [('A', 'B', 1), ('B', 'C', 1)]

edgearray = [('A', 'B', 1), ('A', 'C', 1), ('B', 'A', 1), ('B', 'C', 3), ('D', 'B', 6), ('C', 'D', 4), ('E', 'D', 2), ('E', 'F', 3), ('C', 'G', 1), ('A', 'H', 1), ('G', 'H', 3)]

# Select start node and a target_node here. Input their names as a string.
# If you do not the shortest path to be shown, then leave it empty by typing "target_node = None"
start_node = 'E'
target_node = 'H'

create_graph(nodearray, edgearray)
dijkstra(start_node)
show_shortest_path(start_node, target_node)
reset_graph()

Currently visiting E
	Updated distance of node D from previously inf to 2
	Updated distance of node F from previously inf to 3
	Node E has been finished

Currently visiting D
	Updated distance of node B from previously inf to 8
	Node D has been finished

Currently visiting F
	Node F has been finished

Currently visiting B
	Updated distance of node A from previously inf to 9
	Updated distance of node C from previously inf to 11
	Node B has been finished

Currently visiting A
	Updated distance of node C from previously 11 to 10
	Updated distance of node H from previously inf to 10
	Node A has been finished

Currently visiting C
	Updated distance of node G from previously inf to 11
	Node C has been finished

Currently visiting H
	Node H has been finished

Currently visiting G
	Node G has been finished

The algorithm is over
The nodes were visited in the following order:
 ['E', 'D', 'F', 'B', 'A', 'C', 'H', 'G']
Here is each node's data:

	Name: A
	Predecessor: B
	Distance: 9 

	Name: B
	P

## Yet another example

In [10]:
nodearray = ['A', 'B', 'C', 'D', 'E']

edgearray = [('A', 'D', 3), ('A', 'C', 22), ('B', 'A', 12), ('B', 'C', 5), ('B', 'D', 6), ('C', 'A', 3), ('C', 'E', 5), 
             ('D', 'A', 3), ('D', 'C', 8), ('D', 'E', 18), ('E', 'B', 7), ('E', 'C', 6)]

start_node = 'A'
target_node = 'B'

create_graph(nodearray, edgearray)
dijkstra(start_node)
show_shortest_path(start_node, target_node)
reset_graph()

Currently visiting A
	Updated distance of node C from previously inf to 22
	Updated distance of node D from previously inf to 3
	Node A has been finished

Currently visiting D
	Updated distance of node C from previously 22 to 11
	Updated distance of node E from previously inf to 21
	Node D has been finished

Currently visiting C
	Updated distance of node E from previously 21 to 16
	Node C has been finished

Currently visiting E
	Updated distance of node B from previously inf to 23
	Node E has been finished

Currently visiting B
	Node B has been finished

The algorithm is over
The nodes were visited in the following order:
 ['A', 'D', 'C', 'E', 'B']
Here is each node's data:

	Name: A
	Predecessor: NIL
	Distance: 0 

	Name: B
	Predecessor: E
	Distance: 23 

	Name: C
	Predecessor: D
	Distance: 11 

	Name: D
	Predecessor: A
	Distance: 3 

	Name: E
	Predecessor: C
	Distance: 16 

The path to get from A to B is:
A --> D --> C --> E --> B


## Your tests go here...

In [11]:
nodearray = ['A', 'B', 'C', 'D']

edgearray = [('A', 'C', 3), ('A', 'B', 9), ('A', 'D', 10), ('B', 'A', 8), ('B', 'D', 1), ('C', 'B', 5)]

start_node = 'A'
target_node = 'D'

create_graph(nodearray, edgearray)
dijkstra(start_node)
show_shortest_path(start_node, target_node)
reset_graph()

Currently visiting A
	Updated distance of node B from previously inf to 9
	Updated distance of node C from previously inf to 3
	Updated distance of node D from previously inf to 10
	Node A has been finished

Currently visiting C
	Updated distance of node B from previously 9 to 8
	Node C has been finished

Currently visiting B
	Updated distance of node D from previously 10 to 9
	Node B has been finished

Currently visiting D
	Node D has been finished

The algorithm is over
The nodes were visited in the following order:
 ['A', 'C', 'B', 'D']
Here is each node's data:

	Name: A
	Predecessor: NIL
	Distance: 0 

	Name: B
	Predecessor: C
	Distance: 8 

	Name: C
	Predecessor: A
	Distance: 3 

	Name: D
	Predecessor: B
	Distance: 9 

The path to get from A to D is:
A --> C --> B --> D
